In [4]:
import sqlite3
import pandas as pd 
from ripe.atlas.sagan.helpers import abuf
import base64

In [5]:
conn = sqlite3.connect("data/ripe_results_txt.db")

In [6]:
def prepare_data(version):
    df_results = pd.read_sql(f"""SELECT r.msm_id, r.prb_id, r.sub_id, r.rt, r.dst_addr, r.udp_size as frontend_udp_size, r.proto as proto, r.abuf,
                                        m.use_probe_resolver,
                                        t.flags, t.header, t.udp_size as backend_udp_size, t.proto as backend_proto, t.backend_resolver, t.question,
                                        pr.asn_v4, pr.continent_code, pr.country_code,
                                        a.number, a.name as as_name
                                 from measurements m
                                 JOIN results r ON m.id = r.msm_id
                                 LEFT JOIN txt_results t ON t.prb_id = r.prb_id and
                                                            t.msm_id = r.msm_id and
                                                            t.sub_id = r.sub_id
                                  JOIN probes pr ON r.prb_id = pr.id
                                  JOIN autonomous_systems a ON a.number = pr.asn_v4
                                WHERE r.af = {version}""",conn)
    return df_results 

In [7]:
data_v4 = prepare_data(4)

In [8]:
data_head = data_v4.head(10000)

In [9]:
options = []
def decode_abuf(row):
    data_abuf = row["abuf"]
    try:
        test = abuf.AbufParser.parse(base64.b64decode(data_abuf))
        if test is not None and "EDNS0" in test: 
            if "Option" in test["EDNS0"] and len(test["EDNS0"]["Option"]) > 0:
                return test["EDNS0"]["Option"][0]["OptionCode"]
            else:
                return None
        else: 
            return None
    except: 
        return None
        

In [14]:
data_v4["Option"] = data_v4.apply(decode_abuf, axis=1)

In [19]:
df = data_v4[~data_v4["Option"].isnull()]

In [31]:
df

,msm_id,prb_id,sub_id,rt,dst_addr,frontend_udp_size,proto,abuf,use_probe_resolver,flags,...,backend_udp_size,backend_proto,backend_resolver,question,asn_v4,continent_code,country_code,number,as_name,Option
214,34856460,13040,1,183.436,1.1.1.1,1232,UDP,hnuBgAABAAEAAAABBTEzMDQwCjE2NDI1ODM3NjgLbWVtYn...,None,,...,1452.0,UDP,2400:cb00:19:1024::8d65:4286,13040.1642583768.membrain-it.technology. IN TXT,12322,EU,FR,12322,PROXAD,15.0
237,34856460,13407,1,160.614,1.1.1.1,1232,UDP,72OBgAABAAEAAAABBTEzNDA3CjE2NDI1ODM3NjgLbWVtYn...,None,,...,1452.0,UDP,2400:cb00:19:1024::8d65:425a,13407.1642583768.membrain-it.technology. IN TXT,12322,EU,FR,12322,PROXAD,15.0
560,34856460,18915,1,153.335,1.1.1.1,1232,UDP,sH+BgAABAAEAAAABBTE4OTE1CjE2NDI1ODM3NjcLbWVtYn...,None,,...,1452.0,UDP,2400:cb00:19:1024::8d65:4286,18915.1642583767.membrain-it.technology. IN TXT,3215,EU,FR,3215,AS3215,15.0
1228,34856460,31438,1,952.181,1.1.1.1,1232,UDP,AA2BgAABAAEAAAABBTMxNDM4CjE2NDI1ODM3NjgLbWVtYn...,None,,...,1452.0,UDP,2400:cb00:129:1024::ac44:f125,31438.1642583768.membrain-it.technology. IN TXT,17552,AS,TH,17552,"TRUE-AS-AP True Internet Co.,Ltd.",15.0
1248,34856460,31868,1,98.343,1.1.1.1,1232,UDP,UC6BgAABAAEAAAABBTMxODY4CjE2NDI1ODM3NjgLbWVtYn...,None,,...,1452.0,UDP,2400:cb00:71:1024::a29e:531f,31868.1642583768.membrain-it.technology. IN TXT,9145,EU,DE,9145,EWETEL Cloppenburger Strasse 310,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3527067,34961523,31874,1,65.301,192.168.179.1,1232,UDP,+2CBgAABAAEAAAABBTMxODc0CjE2NDMxNTE3NjYLbWVtYn...,True,,...,1452.0,UDP,172.68.109.25,31874.1643151766.membrain-it.technology. IN TXT,8767,EU,DE,8767,MNET-AS Germany,12.0
3527114,34961523,33902,2,39.71,172.18.4.1,1232,UDP,ACqBgAABAAEAAAABBTMzOTAyCjE2NDMxNTE3NjgLbWVtYn...,True,,...,1452.0,UDP,172.70.241.63,33902.1643151768.membrain-it.technology. IN TXT,43341,EU,DE,43341,MDLINK MDlink online service center GmbH,12.0
3527136,34961523,35698,1,118.902,172.21.0.1,1232,UDP,AwSBgAABAAEAAAABBTM1Njk4CjE2NDMxNTE3NjYLbWVtYn...,True,cd,...,1232.0,UDP,192.145.127.148,35698.1643151766.membrain-it.technology. IN TXT,49367,EU,IT,49367,ASSEFLOW Amsterdam Internet Exchange (AMS-IX),12.0
3527457,34961523,54300,1,574.715,192.168.179.1,1232,UDP,t/SBgAABAAEAAAABBTU0MzAwCjE2NDMxNTE4MTQLbWVtYn...,True,cd,...,1232.0,UDP,95.179.134.211,54300.1643151814.membrain-it.technology. IN TXT,206238,EU,NL,206238,FREEDOMINTERNET,12.0


In [32]:
df["Option"].unique()

array([15., 12.])

In [20]:
v = df["abuf"].values[0]
test = abuf.AbufParser.parse(base64.b64decode(v))
print(test)

{'HEADER': {'ID': 34427, 'QR': True, 'OpCode': 'QUERY', 'AA': False, 'TC': False, 'RD': True, 'RA': True, 'Z': 0, 'AD': False, 'CD': False, 'ReturnCode': 'NOERROR', 'QDCOUNT': 1, 'ANCOUNT': 1, 'NSCOUNT': 0, 'ARCOUNT': 1}, 'QuestionSection': [{'Qname': '13040.1642583768.membrain-it.technology.', 'Qtype': 'TXT', 'Qclass': 'IN'}], 'AnswerSection': [{'Name': '13040.1642583768.membrain-it.technology.', 'Type': 'TXT', 'Class': 'IN', 'TTL': 3600, 'RDlength': 279, 'Data': ['FROM_2400:cb00:19:1024::8d65:4286', 'Protocol_UDP', 'opcode:&QUERY,&status:&NOERROR,&id:&48496$', 'flags:%&QUERY:&1,&ANSWER:&0,&AUTHORITY:&0,&ADDITIONAL:&1$$', 'OPT&PSEUDOSECTION:$%&EDNS:&version&0%&flags:&do%&udp:&1452$$', 'QUESTION&SECTION:$%13040.1642583768.membrain-it.technology.?IN?&TXT$']}], 'EDNS0': {'UDPsize': 1232, 'ExtendedReturnCode': 0, 'Version': 0, 'Z': 0, 'Type': 'OPT', 'Option': [{'OptionCode': 15, 'OptionLength': 76}], 'Name': '.'}}


In [28]:
def decode_abuf_2(row):
    data_abuf = row["abuf"]
    try:
        test = base64.b64decode(data_abuf)
        test_parsed = abuf.AbufParser.parse(test)
        print(test)
        print(test_parsed)
    except: 
        i = 0 

In [29]:
data_head = data_v4.head(100)

In [30]:
data_head.apply(decode_abuf_2, axis=1)

b'H\x82\x81\x80\x00\x01\x00\x01\x00\x00\x00\x01\x0512894\n1642583757\x0bmembrain-it\ntechnology\x00\x00\x10\x00\x01\xc0\x0c\x00\x10\x00\x01\x00\x00\x0e\x10\x01\x16 FROM_2400:cb00:47:1024::a29e:11c\x0cProtocol_UDP*opcode:&QUERY,&status:&NOERROR,&id:&58276$:flags:%&QUERY:&1,&ANSWER:&0,&AUTHORITY:&0,&ADDITIONAL:&1$$<OPT&PSEUDOSECTION:$%&EDNS:&version&0%&flags:&do%&udp:&1452$$DQUESTION&SECTION:$%12894.1642583757.membrain-it.technology.?IN?&TXT$\x00\x00)\x04\xd0\x00\x00\x00\x00\x00\x00'
{'HEADER': {'ID': 18562, 'QR': True, 'OpCode': 'QUERY', 'AA': False, 'TC': False, 'RD': True, 'RA': True, 'Z': 0, 'AD': False, 'CD': False, 'ReturnCode': 'NOERROR', 'QDCOUNT': 1, 'ANCOUNT': 1, 'NSCOUNT': 0, 'ARCOUNT': 1}, 'QuestionSection': [{'Qname': '12894.1642583757.membrain-it.technology.', 'Qtype': 'TXT', 'Qclass': 'IN'}], 'AnswerSection': [{'Name': '12894.1642583757.membrain-it.technology.', 'Type': 'TXT', 'Class': 'IN', 'TTL': 3600, 'RDlength': 278, 'Data': ['FROM_2400:cb00:47:1024::a29e:11c', 'Protoc

0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Length: 100, dtype: object